# Imports

In [18]:
import gmsh
import os
# Limpar o terminal
os.system('cls' if os.name == 'nt' else 'clear')

0

# Mesh Parameters

In [19]:
FINITE_ELEMENT = ("Tetrahedron", 1)
BOUNDARY = [{'tag': 201, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}]
MATERIAL = [{'tag': 301, 'name': 'free_space', 'ur': 1, 'er': 1}]
element, order = FINITE_ELEMENT
h = 1  # Tamanho do elemento da malha

# Geometrical Data

In [20]:
# Dimensões da cavidade retangular, metros
# a, b, c = 1.0, 0.6, 0.4
a, b, c = 1.0, 1.0, 1.0

# Gmsh Initialize

In [21]:
gmsh.initialize()
gmsh.model.add("rectangular_cavity")

## Define domain

In [22]:
rectangular_cavity = gmsh.model.occ.addBox(0, 0, 0, a, b, c)
print(f"\nrectangular_cavity_Tag: {rectangular_cavity}")
gmsh.model.occ.synchronize()


rectangular_cavity_Tag: 1


## Mesh Size 

In [23]:
# gmsh.option.setNumber("Mesh.MeshSizeMax", h)
gmsh.option.setNumber("Mesh.MeshSizeMin", h)
gmsh.model.mesh.generate(dim=3)
gmsh.model.mesh.setOrder(order)

## 1. _Edge mapping_

In [24]:
gmsh.model.mesh.createEdges()
edgeTags, edgeNodes = gmsh.model.mesh.getAllEdges()
print(f"Total number of edges: {len(edgeTags)}")
print(f"Total number of edgeNodes: {len(edgeNodes)}")

Total number of edges: 49
Total number of edgeNodes: 98


In [25]:
# Criar edge_mapping
edge_mapping = {tuple(sorted([edgeNodes[2*i], edgeNodes[2*i + 1]])): tag
                 for i, tag in enumerate(sorted(edgeTags))}
print(f"edge_mapping:\n {edge_mapping}")

edge_mapping:
 {(9, 13): 1, (9, 12): 2, (9, 11): 3, (9, 14): 4, (12, 14): 5, (10, 13): 6, (11, 13): 7, (11, 12): 8, (1, 14): 9, (10, 12): 10, (8, 13): 11, (10, 14): 12, (6, 13): 13, (5, 14): 14, (4, 9): 15, (11, 14): 16, (5, 10): 17, (2, 13): 18, (3, 7): 19, (4, 8): 20, (8, 10): 21, (3, 14): 22, (6, 8): 23, (5, 6): 24, (2, 4): 25, (2, 9): 26, (12, 13): 27, (2, 6): 28, (3, 12): 29, (1, 3): 30, (6, 10): 31, (2, 11): 32, (4, 12): 33, (1, 9): 34, (5, 7): 35, (7, 8): 36, (3, 4): 37, (1, 2): 38, (6, 11): 39, (4, 13): 40, (1, 5): 41, (3, 9): 42, (10, 11): 43, (8, 12): 44, (7, 10): 45, (7, 14): 46, (1, 11): 47, (5, 11): 48, (7, 12): 49}


## 2. _Face mapping_

In [26]:
gmsh.model.mesh.createFaces()
faceTags, faceNodes = gmsh.model.mesh.getAllFaces(3)
print(f"Total number of faces: {len(faceTags)}")
print(f"Total number of faceNodes: {len(faceNodes)}")

Total number of faces: 60
Total number of faceNodes: 180


In [27]:
# Criar face_mapping
face_mapping = {tuple(sorted([faceNodes[3*i], faceNodes[3*i + 1], faceNodes[3*i + 2]])): tag
                 for i, tag in enumerate(sorted(faceTags))}
print(f"face_mapping:\n {face_mapping}")

face_mapping:
 {(6, 10, 13): 1, (11, 12, 14): 2, (5, 11, 14): 3, (8, 12, 13): 4, (3, 7, 14): 5, (7, 10, 12): 6, (10, 11, 12): 7, (1, 3, 9): 8, (5, 7, 14): 9, (3, 7, 12): 10, (6, 8, 13): 11, (2, 9, 11): 12, (6, 8, 10): 13, (1, 5, 14): 14, (10, 11, 13): 15, (2, 4, 13): 16, (3, 4, 12): 17, (4, 8, 12): 18, (5, 6, 10): 19, (10, 12, 13): 20, (8, 10, 12): 21, (2, 4, 9): 22, (7, 12, 14): 23, (1, 3, 14): 24, (2, 6, 13): 25, (5, 7, 10): 26, (4, 12, 13): 27, (7, 8, 12): 28, (11, 12, 13): 29, (3, 4, 9): 30, (2, 6, 11): 31, (1, 2, 11): 32, (1, 2, 9): 33, (7, 8, 10): 34, (1, 5, 11): 35, (5, 6, 11): 36, (10, 11, 14): 37, (1, 11, 14): 38, (8, 10, 13): 39, (9, 12, 14): 40, (6, 11, 13): 41, (9, 11, 14): 42, (4, 9, 12): 43, (9, 11, 12): 44, (2, 9, 13): 45, (9, 11, 13): 46, (3, 9, 12): 47, (1, 9, 14): 48, (5, 10, 14): 49, (4, 8, 13): 50, (1, 9, 11): 51, (3, 12, 14): 52, (10, 12, 14): 53, (5, 10, 11): 54, (6, 10, 11): 55, (9, 12, 13): 56, (2, 11, 13): 57, (3, 9, 14): 58, (7, 10, 14): 59, (4, 9, 13): 60}


## 3. Applying _Physical Groups_

In [28]:
# Obter os contornos (superfícies, dim=2) do volume
BoundaryDimTags = gmsh.model.getBoundary([(3, rectangular_cavity)], oriented=True, recursive=False)
print(f"BoundaryDimTags: {BoundaryDimTags}")

BoundaryDimTags: [(2, -1), (2, 2), (2, -3), (2, 4), (2, -5), (2, 6)]


In [29]:
# Exibir os TAGs das superfícies associadas a cada contorno
BoundaryTags = [Dimtags[1] for Dimtags in BoundaryDimTags]

# Definindo as superfícies de contorno de Dirichlet (dim=2)
gmsh.model.addPhysicalGroup(dim=2, tags=BoundaryTags, tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])

# Adicionar grupos físicos para Dim=3 (volume)
gmsh.model.addPhysicalGroup(dim=3, tags=[rectangular_cavity], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])

301

## 4. Mesh Elements by Dimension
### Dimension 0 (1-node point)

In [30]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=0)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"len(elemTag): {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"len(nodeTag): {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [15] (1-node point)
elemTags: [array([1, 2, 3, 4, 5, 6, 7, 8], dtype=uint64)]
len(elemTag): [8]
elemNodeTags: [array([1, 2, 3, 4, 5, 6, 7, 8], dtype=uint64)]
len(nodeTag): [8]


### Dimension 1 (2-node line)

In [31]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=1)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"len(elemTag): {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"len(nodeTag): {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [1] (2-node line)
elemTags: [array([ 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], dtype=uint64)]
len(elemTag): [12]
elemNodeTags: [array([2, 1, 1, 3, 4, 3, 2, 4, 6, 5, 5, 7, 8, 7, 6, 8, 2, 6, 1, 5, 4, 8,
       3, 7], dtype=uint64)]
len(nodeTag): [24]


### Dimension 2 (3-node triangle)

In [32]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=2)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"len(elemTag): {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"len(nodeTag): {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [2] (3-node triangle)
elemTags: [array([21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44], dtype=uint64)]
len(elemTag): [24]
elemNodeTags: [array([ 2,  1,  9,  1,  3,  9,  4,  2,  9,  3,  4,  9,  6, 10,  5,  5, 10,
        7,  8, 10,  6,  7, 10,  8,  1,  2, 11,  5,  1, 11,  2,  6, 11,  6,
        5, 11,  3, 12,  4,  7, 12,  3,  4, 12,  8,  8, 12,  7,  2,  4, 13,
        6,  2, 13,  4,  8, 13,  8,  6, 13,  1, 14,  3,  5, 14,  1,  3, 14,
        7,  7, 14,  5], dtype=uint64)]
len(nodeTag): [72]


### Dimension 3 (4-node tetrahedron)

In [33]:
elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=3)
print(f"elemTypes: {elementTypes}")
print(f"elemTags: {elementTags}")
print(f"len(elemTag): {[len(elementTag) for elementTag in elementTags]}")
print(f"elemNodeTags: {nodeTags}")
print(f"len(nodeTag): {[len(nodeTag) for nodeTag in nodeTags]}")

elemTypes: [4] (4-node tetrahedron)
elemTags: [array([45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
       62, 63, 64, 65, 66, 67, 68], dtype=uint64)]
len(elemTag): [24]
elemNodeTags: [array([11, 12, 10, 13, 11, 12, 14, 10, 14, 12, 11,  9,  9, 12, 11, 13, 11,
       14,  1,  5,  4, 12,  3,  9,  3, 14,  1,  9,  1, 11,  2,  9, 12,  3,
       14,  7,  8, 12,  4, 13, 10,  5, 11,  6, 14,  5, 10,  7,  7, 10, 12,
        8,  2,  4,  9, 13, 11,  2, 13,  6, 13,  8, 10,  6, 14,  3, 12,  9,
        5, 11, 14, 10,  1, 14, 11,  9,  8, 10, 12, 13,  4, 12,  9, 13,  9,
       11,  2, 13, 12, 14, 10,  7, 13, 10, 11,  6], dtype=uint64)]
len(nodeTag): [96]


## 5. Geometrical Entities by Dimension
dimTags (vector of pairs of integers)
### Dimension 0 (Points)

In [ ]:
gmsh.model.getEntities(dim=0)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]

### Dimension 1 (Edges)

gmsh.model.getEntities(dim=1)

### Dimension 2 (Surfaces)

In [38]:
gmsh.model.getEntities(dim=2)

[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6)]

### Dimension 3 (Regions)

In [39]:
gmsh.model.getEntities(dim=3)

[(3, 1)]

## 6. ``mesh_data['cell']``: Material Physical Groups (dim=3)

In [40]:
cell_data = {}
for material in MATERIAL:
    # Obter as entidades físicas (grupo físico) associadas ao material
    MaterialEntitiesTags = gmsh.model.getEntitiesForPhysicalGroup(dim=3, tag=material['tag'])
    print(f"Entities of {material['name']} with dim3: {MaterialEntitiesTags}")
    
    for EntityTag in MaterialEntitiesTags:
        elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=3, tag=EntityTag)
        print(f"mesh_elements of entity (dim, tag) = (3, {EntityTag}): elemTags = {elementTags}")
        print(f"mesh_elements of entity (dim, tag) = (3, {EntityTag}): nodeTags = {nodeTags}")
        
        for elemType, elemTag, elemNode in zip(elementTypes, elementTags, nodeTags):            
            # Obter as propriedades do elemento
            _, _, _, nodes_per_element, _, _ = gmsh.model.mesh.getElementProperties(elemType)

            # Número de elementos
            N_tet = len(elemNode) // nodes_per_element

            # Criar dicionário associando cada elemento à sua conectividade de nós
            for i, Tag in enumerate(elemTag):                
                # Obter a conectividade do elemento
                # conn_node = list(sorted(elemNodes[0][4 * i : 4 * (i + 1)])) 
                conn_node = elemNode[nodes_per_element * i: nodes_per_element * (i + 1)].tolist()
                conn_std = sorted(conn_node)

                # Tetrahedron element
                if nodes_per_element == 4:
                    conn_edge = [
                        edge_mapping[(conn_std[0], conn_std[1])],  # e1: 1 -> 2
                        edge_mapping[(conn_std[0], conn_std[2])],  # e2: 1 -> 3
                        edge_mapping[(conn_std[0], conn_std[3])],  # e3: 1 -> 4
                        edge_mapping[(conn_std[1], conn_std[2])],  # e4: 2 -> 3
                        edge_mapping[(conn_std[1], conn_std[3])],  # e5: 2 -> 4
                        edge_mapping[(conn_std[2], conn_std[3])]   # e6: 3 -> 4
                    ]

                    # Conectividade de faces (cada face tem 3 nós)
                    conn_face = [
                        sorted([conn_node[0], conn_node[1], conn_node[2]]),  # Face 1
                        sorted([conn_node[0], conn_node[1], conn_node[3]]),  # Face 2
                        sorted([conn_node[0], conn_node[2], conn_node[3]]),  # Face 3
                        sorted([conn_node[1], conn_node[2], conn_node[3]])   # Face 4
                    ]
                
                # Adicionar ao dicionário de células
                cell_data[Tag] = {
                    'tag': Tag,
                    'conn': conn_node,
                    'conn_sorted': conn_std,
                    'conn_edge': conn_edge,
                    'conn_face': conn_face,
                    # 'geo': {'centroid': None, 'dim': None},
                    # 'contour': {'type': None, 'conn_contour': None},
                    'material': material['tag']}

Entities of free_space with dim3: [1]
mesh_elements of entity (dim, tag) = (3, 1): elemTags = [array([45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
       62, 63, 64, 65, 66, 67, 68], dtype=uint64)]
mesh_elements of entity (dim, tag) = (3, 1): nodeTags = [array([11, 12, 10, 13, 11, 12, 14, 10, 14, 12, 11,  9,  9, 12, 11, 13, 11,
       14,  1,  5,  4, 12,  3,  9,  3, 14,  1,  9,  1, 11,  2,  9, 12,  3,
       14,  7,  8, 12,  4, 13, 10,  5, 11,  6, 14,  5, 10,  7,  7, 10, 12,
        8,  2,  4,  9, 13, 11,  2, 13,  6, 13,  8, 10,  6, 14,  3, 12,  9,
        5, 11, 14, 10,  1, 14, 11,  9,  8, 10, 12, 13,  4, 12,  9, 13,  9,
       11,  2, 13, 12, 14, 10,  7, 13, 10, 11,  6], dtype=uint64)]


In [46]:
# Adiciona nova chave sequencial ao cell_data
cell_data = {i+1: cell_data[Tag] for i, Tag in enumerate(cell_data)}; cell_data

{1: {'tag': 45,
  'conn': [11, 12, 10, 13],
  'conn_sorted': [10, 11, 12, 13],
  'conn_edge': [43, 10, 6, 8, 7, 27],
  'conn_face': [[10, 11, 12], [11, 12, 13], [10, 11, 13], [10, 12, 13]],
  'material': 301},
 2: {'tag': 46,
  'conn': [11, 12, 14, 10],
  'conn_sorted': [10, 11, 12, 14],
  'conn_edge': [43, 10, 12, 8, 16, 5],
  'conn_face': [[11, 12, 14], [10, 11, 12], [10, 11, 14], [10, 12, 14]],
  'material': 301},
 3: {'tag': 47,
  'conn': [14, 12, 11, 9],
  'conn_sorted': [9, 11, 12, 14],
  'conn_edge': [3, 2, 4, 8, 16, 5],
  'conn_face': [[11, 12, 14], [9, 12, 14], [9, 11, 14], [9, 11, 12]],
  'material': 301},
 4: {'tag': 48,
  'conn': [9, 12, 11, 13],
  'conn_sorted': [9, 11, 12, 13],
  'conn_edge': [3, 2, 1, 8, 7, 27],
  'conn_face': [[9, 11, 12], [9, 12, 13], [9, 11, 13], [11, 12, 13]],
  'material': 301},
 5: {'tag': 49,
  'conn': [11, 14, 1, 5],
  'conn_sorted': [1, 5, 11, 14],
  'conn_edge': [41, 47, 9, 48, 14, 16],
  'conn_face': [[1, 11, 14], [5, 11, 14], [1, 5, 11], [1, 

## 7. `mesh_data['boundary']`: Boundary Physical Groups (dim=2)

In [42]:
boundary_data = {}
for bc in BOUNDARY:
    # Obter as entidades físicas (grupo físico) associadas ao contorno
    BoundaryEntitiesTags = gmsh.model.getEntitiesForPhysicalGroup(dim=2, tag=bc['tag'])
    print(f"Entities of {bc['name']} with dim2: {BoundaryEntitiesTags}")
    
    for EntityTag in BoundaryEntitiesTags:
        elementTypes, elementTags, nodeTags = gmsh.model.mesh.getElements(dim=2, tag=EntityTag)
        print(f"mesh_elements of entity (dim, tag) = (2, {EntityTag}): elemTags = {elementTags}")
        print(f"mesh_elements of entity (dim, tag) = (2, {EntityTag}): nodeTags = {nodeTags}")
        
        for elemType, elemTag, elemNode in zip(elementTypes, elementTags, nodeTags):   
            # Obter as propriedades do elemento
            _, _, _, nodes_per_element, _, _ = gmsh.model.mesh.getElementProperties(elemType)

            # Número de elementos
            N_tri = len(elemNode) // nodes_per_element             

            # Criar dicionário associando cada elemento à sua conectividade de nós
            for i, Tag in enumerate(elemTag):    
                # Obter a conectividade do elemento            
                # conn_node = list(sorted(nodeTags[0][3 * i : 3 * (i + 1)]))
                conn_node = elemNode[nodes_per_element * i: nodes_per_element * (i + 1)].tolist()
                conn_std = sorted(conn_node)

                # Triangular element
                if nodes_per_element == 3:
                    conn_edge = [
                        edge_mapping[(conn_std[0], conn_std[1])],  # e1: 1 -> 2
                        edge_mapping[(conn_std[0], conn_std[2])],  # e2: 1 -> 3
                        edge_mapping[(conn_std[1], conn_std[2])],  # e3: 2 -> 3
                    ]

                # Dicionário de contorno
                boundary_data[Tag] = {
                    'tag': Tag,
                    'conn': conn_node,
                    'conn_sorted': conn_std,
                    'conn_edge': conn_edge,
                    'boundary': bc['tag']}

Entities of nxE=0 with dim2: [1 2 3 4 5 6]
mesh_elements of entity (dim, tag) = (2, 1): elemTags = [array([21, 22, 23, 24], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 1): nodeTags = [array([2, 1, 9, 1, 3, 9, 4, 2, 9, 3, 4, 9], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 2): elemTags = [array([25, 26, 27, 28], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 2): nodeTags = [array([ 6, 10,  5,  5, 10,  7,  8, 10,  6,  7, 10,  8], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 3): elemTags = [array([29, 30, 31, 32], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 3): nodeTags = [array([ 1,  2, 11,  5,  1, 11,  2,  6, 11,  6,  5, 11], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 4): elemTags = [array([33, 34, 35, 36], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 4): nodeTags = [array([ 3, 12,  4,  7, 12,  3,  4, 12,  8,  8, 12,  7], dtype=uint64)]
mesh_elements of entity (dim, tag) = (2, 5): elemTags = [array([37, 38, 3

In [47]:
# Adiciona nova chave sequencial ao boundary_data
boundary_data = {i+1: boundary_data[Tag] for i, Tag in enumerate(boundary_data)}; boundary_data

{1: {'tag': 21,
  'conn': [2, 1, 9],
  'conn_sorted': [1, 2, 9],
  'conn_edge': [38, 34, 26],
  'boundary': 201},
 2: {'tag': 22,
  'conn': [1, 3, 9],
  'conn_sorted': [1, 3, 9],
  'conn_edge': [30, 34, 42],
  'boundary': 201},
 3: {'tag': 23,
  'conn': [4, 2, 9],
  'conn_sorted': [2, 4, 9],
  'conn_edge': [25, 26, 15],
  'boundary': 201},
 4: {'tag': 24,
  'conn': [3, 4, 9],
  'conn_sorted': [3, 4, 9],
  'conn_edge': [37, 42, 15],
  'boundary': 201},
 5: {'tag': 25,
  'conn': [6, 10, 5],
  'conn_sorted': [5, 6, 10],
  'conn_edge': [24, 17, 31],
  'boundary': 201},
 6: {'tag': 26,
  'conn': [5, 10, 7],
  'conn_sorted': [5, 7, 10],
  'conn_edge': [35, 17, 45],
  'boundary': 201},
 7: {'tag': 27,
  'conn': [8, 10, 6],
  'conn_sorted': [6, 8, 10],
  'conn_edge': [23, 31, 21],
  'boundary': 201},
 8: {'tag': 28,
  'conn': [7, 10, 8],
  'conn_sorted': [7, 8, 10],
  'conn_edge': [36, 45, 21],
  'boundary': 201},
 9: {'tag': 29,
  'conn': [1, 2, 11],
  'conn_sorted': [1, 2, 11],
  'conn_edge'

# Gmsh Finalize

In [ ]:
# Visualizar a malha no ambiente Gmsh (opcional)
# gmsh.fltk.run()

# Finalizar o Gmsh
gmsh.finalize()